<img src="https://geodecode.com.br/wp-content/uploads/2021/12/brasildatacube.png" align="left" width="74"/>
<img src="https://brazil-data-cube.github.io/_images/logo-odc-bdc.png" align="right" width="74" />

<h1 style="color:#336699; text-align: center" >Annual Metting Use Case</h1>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    <sup>
        <a href="https://orcid.org/0000-0002-9916-1167">
            <i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i>
        </a>
    </sup>
    <br/>
    Brazil Data Cube (BDC), National Institute for Space Research (INPE)
    <br/>
    <br/>
    <br/>
    <div style="width: 98%;">
        <div style="text-align: justify; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 10px;">
            <b>Explantion here....:</b>
            <div>
                Write write write write writewrite writewrite write. <a href="http://www.brazildatacube.org/" target="_blank">Site site site site site</a>, <i> Outras infos</i> .......
            </div>
        </div>
    </div>
</div>

In [3]:
library(sf)
library(rstac)
library(dplyr)

Linking to GEOS 3.10.2, GDAL 3.4.1, PROJ 8.2.1; sf_use_s2() is TRUE


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# Set constants
<hr style="border:1px solid #0077b9;">

In [4]:
HARMONIZE_STAC_URL <- 'https://brazildatacube.dpi.inpe.br/harmonize/dev/stac/v1/'

FINAL_CSV_PREFIX <- "data_36_months_"

START_DATE <- "2019-01-01"
END_DATE <- "2021-12-31"

# List Harmonize STAC collections (using RSTAC)
<hr style="border:1px solid #0077b9;">

In [5]:
all_collections <- stac(HARMONIZE_STAC_URL) %>%
  collections() %>%
  get_request()

for (collection in all_collections$collections){
  print(collection$id)
}

[1] "dengue_mortality_rate_mun_year_north-1"
[1] "max_temper_mean_ne_mun_month-1"
[1] "total_prec_max_no_mun_epiweek-1"
[1] "Phantom3Advanced_FlightHeight120m_Mosaic-1"
[1] "Phantom3Advanced_FlightHeight120m-1"
[1] "chagas_disease_cases_mun_week_north-1"
[1] "max_temper_mean_no_mun_month-1"
[1] "max_temper_max_no_mun_month-1"
[1] "max_temper_max_no_mun_epiweek_ras-1"
[1] "chagas_disease_cases_mun_month_north-1"
[1] "max_temper_max_ne_mun_epiweek-1"
[1] "dengue_cases_mun_year_north-1"
[1] "chikungunya_cases_mun_year_north-1"
[1] "zika_cases_mun_month_north-1"
[1] "dengue_cases_mun_month_north-1"
[1] "chikungunya_cases_mun_month_north-1"
[1] "dengue_cases_mun_week_north-1"
[1] "dengue_mortality_rate_mun_month_north-1"
[1] "max_temper_mean_no_mun_epiweek-1"
[1] "zika_cases_mun_year_north-1"
[1] "chagas_disease_cases_mun_year_north-1"
[1] "zika_cases_mun_week_north-1"
[1] "total_prec_max_no_mun_month-1"
[1] "dengue_mortality_rate_mun_week_north-1"
[1] "chikungunya_cases_mun_week_north-1"
[

# Obtain montly Temperature data for each municipality
<hr style="border:1px solid #0077b9;">

### Set collection id

In [6]:
TEMPERATURE_COLLECTION_ID <- "max_temper_mean_no_mun_month-1"

### Search Temperature data on Harmonize STAC (Using RSTAC)

In [7]:
temperature_search <- stac(HARMONIZE_STAC_URL) %>%
  stac_search(
    collections = TEMPERATURE_COLLECTION_ID,
    datetime = paste(START_DATE, END_DATE, sep = "/"),
    limit = 50
  ) %>%
  get_request()

temperature_search$context

$matched
[1] 36

$returned
[1] 36

### For each feature, retrieve geojson as dataframe

In [8]:
dengue_feature_to_geojson_csv <- function(feature) {
  datetime <- feature$properties$datetime
  geojson_href <- feature$assets$geojson$href

  df <- as.data.frame(st_read(geojson_href)) %>%
    dplyr::select(CD_MUN, NM_MUN, SIGLA, value) %>%
      rename(cd_mun = CD_MUN, nm_mun = NM_MUN, UF = SIGLA, temperature = value) %>%
      mutate(year_month = format(as.Date(datetime), "%Y-%m"))
}

temperature_dfs <- lapply(X = temperature_search$features, FUN = dengue_feature_to_geojson_csv)
temperature_dfs[1:2]

Reading layer `max_temper_mean_NO_mun_month_20211201' from data source 
  `https://brazildatacube.dpi.inpe.br/harmonize/dev/data/climate_indicators/temperature_era5land/max_temper_mean_no_mun_month/shapefiles/2021-12-01/max_temper_mean_no_mun_month_20211201.geojson' 
  using driver `GeoJSON'
Simple feature collection with 21 features and 6 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -50.9967 ymin: -5.196937 xmax: -48.13749 ymax: -1.019427
Geodetic CRS:  WGS 84
Reading layer `max_temper_mean_NO_mun_month_20211101' from data source 
  `https://brazildatacube.dpi.inpe.br/harmonize/dev/data/climate_indicators/temperature_era5land/max_temper_mean_no_mun_month/shapefiles/2021-11-01/max_temper_mean_no_mun_month_20211101.geojson' 
  using driver `GeoJSON'
Simple feature collection with 21 features and 6 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -50.9967 ymin: -5.196937 xmax: -48.13749 ymax: -1.019427
Geodetic CRS:  WGS 84
Reading layer `max_t

cd_mun,nm_mun,UF,temperature,year_month
<chr>,<chr>,<chr>,<dbl>,<chr>
1500107,Abaetetuba,PA,32.29361,2021-12
1500800,Ananindeua,PA,32.45644,2021-12
1501105,Bagre,PA,32.98824,2021-12
1501204,Baião,PA,32.64342,2021-12
1501303,Barcarena,PA,32.06986,2021-12
1501402,Belém,PA,32.09974,2021-12
1501501,Benevides,PA,32.67789,2021-12
1501782,Breu Branco,PA,33.07226,2021-12
1502103,Cametá,PA,32.05707,2021-12


### Rbind lists of monthly dataframes in one big dataframe

In [9]:
temperature_df <- do.call(rbind, temperature_dfs)

dim(temperature_df)
head(temperature_df)

[1] 756   5

,cd_mun,nm_mun,UF,temperature,year_month
,<chr>,<chr>,<chr>,<dbl>,<chr>
1,1500107,Abaetetuba,PA,32.29361,2021-12
2,1500800,Ananindeua,PA,32.45644,2021-12
3,1501105,Bagre,PA,32.98824,2021-12
4,1501204,Baião,PA,32.64342,2021-12
5,1501303,Barcarena,PA,32.06986,2021-12
6,1501402,Belém,PA,32.09974,2021-12


### Save temperature data

In [10]:
FILENAME <- paste0(FINAL_CSV_PREFIX, "temperature.csv")

write.csv(ndvi_df, FILENAME, row.names = FALSE)